In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [2]:
from load_librairies import *
import xscale.signal.fitting as xsf
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

period_str_1 = 1982
period_end_1 = 1999

period_str_2 = 2000
period_end_2 = 2014



def is_jja(month):
    return (month >= 6) & (month <= 8)
def is_mjj(month):
    return (month >= 5) & (month <= 7)
#def is_amj(month):
#    return (month >= 4) & (month <= 6)

def relative_std(a, b):
    return (a - b) / a

def f_test(x, y):
    x = np.array(x)
    y = np.array(y)
    f = np.var(x, ddof=1)/np.var(y, ddof=1) #calculate F test statistic 
    dfn = x.size-1 #define degrees of freedom numerator 
    dfd = y.size-1 #define degrees of freedom denominator 
    p = 1-stats.f.cdf(f, dfn, dfd) #find p-value of F test statistic 
    return f, p

def month_range_day(start=None, periods=None):
    start_date = pd.Timestamp(start).date()
    month_range = pd.date_range(start=start_date, periods=periods, freq='M')
    month_day = month_range.day.values
    month_day[start_date.day < month_day] = start_date.day
    return pd.to_datetime(month_range.year*10000+month_range.month*100+month_day, format='%Y%m%d')
######################
#      TO CHANGE     # 
######################

path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/new_version/reviews/'
path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

# Observations SST

In [3]:
# 1854-2022
sst_ersst = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERSSTv5/sst.mnmean.ersstv5.nc') 

sst_ersst = xr.concat([sst_ersst.sst[:, :, 90:], sst_ersst.sst[ :,:, :90]], dim='lon')
sst_ersst.coords['lon'] = (sst_ersst.coords['lon'] + 180) % 360 - 180
sst_ersst_1 = sst_ersst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_ersst_1 = sst_ersst_1.chunk(chunks=None)
sst_ersst_1 = xsf.detrend(sst_ersst_1,dim='time',type='linear')


sst_ersst_2 = sst_ersst.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
sst_ersst_2 = sst_ersst_2.chunk(chunks=None)
sst_ersst_2 = xsf.detrend(sst_ersst_2,dim='time',type='linear')

In [4]:
sst_ersst_atl3_1 = Atools.data_sub(sst_ersst_1,-20,0,-3,3).mean(dim='lon').mean(dim='lat')
ssta_ersst_atl3_1,_ = Atools.ano_norm_t(sst_ersst_atl3_1.load())
std_ssta_ersst_atl3_jja_1 = ssta_ersst_atl3_1.sel(time=is_jja(
    ssta_ersst_atl3_1['time.month'])).std(dim='time')

sst_ersst_atl3_2 = Atools.data_sub(sst_ersst_2,-20,0,-3,3).mean(dim='lon').mean(dim='lat')
ssta_ersst_atl3_2,_ = Atools.ano_norm_t(sst_ersst_atl3_2.load())
std_ssta_ersst_atl3_jja_2 = ssta_ersst_atl3_2.sel(time=is_jja(
    ssta_ersst_atl3_2['time.month'])).std(dim='time')

# ERA5 

In [5]:
# 1979-2020
## Load mask ##
nc_mask = xr.open_dataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/ERA_landsea_mask.nc')
lsm = nc_mask.lsm[0,:,:]
lsm = xr.concat([lsm[ :, 720:], lsm[ :, :720]], dim='longitude')
lsm.coords['longitude'] = (lsm.coords['longitude'] + 180) % 360 - 180

### Load data ##
sst_era5_tmp = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/era5.sst.atl.1979.2020.nc')
sst_era5 = sst_era5_tmp.sst[:,0,:,:] - 273.15

## Take the period 1982/01 - 1999/12 ##
sst_era5_1 = sst_era5.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_era5_1 = sst_era5_1.chunk(chunks=None)
sst_era5_1 = xsf.detrend(sst_era5_1,dim='time',type='linear')


## Take the period 2000/01 - 2014/12 ##
sst_era5_2 = sst_era5.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
sst_era5_2 = sst_era5_2.chunk(chunks=None)
sst_era5_2 = xsf.detrend(sst_era5_2,dim='time',type='linear')

sst_era5_atl3_1 = Atools.data_sub(sst_era5_1,-20,0,-3,3).mean(dim='longitude').mean(dim='latitude')
ssta_era5_atl3_1,_ = Atools.ano_norm_t(sst_era5_atl3_1.load())
std_ssta_era5_atl3_jja_1 = ssta_era5_atl3_1.sel(time=is_jja(
    ssta_era5_atl3_1['time.month'])).std(dim='time')

sst_era5_atl3_2 = Atools.data_sub(sst_era5_2,-20,0,-3,3).mean(dim='longitude').mean(dim='latitude')
ssta_era5_atl3_2,_ = Atools.ano_norm_t(sst_era5_atl3_2.load())
std_ssta_era5_atl3_jja_2 = ssta_era5_atl3_2.sel(time=is_jja(
    ssta_era5_atl3_2['time.month'])).std(dim='time')

# OI-SST

In [6]:
# 1981-2019
## Load the data ##
nc = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/OI_SST_v2/oi.sst.v2.monmean.nc')
sst_oiss = nc.sst[:]
sst_oiss = xr.concat([sst_oiss[:, :, 720:], sst_oiss[:, :, :720]], dim='lon')
sst_oiss.coords['lon'] = (sst_oiss.coords['lon'] + 180) % 360 - 180

## Take the period 1982/01-1999/12 ##
sst_oiss_1 = sst_oiss.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_oiss_1 = sst_oiss_1.chunk(chunks=None)
sst_oiss_1 = xsf.detrend(sst_oiss_1,dim='time',type='linear')

## Take the period 2000/01-2014/12 ##
sst_oiss_2 = sst_oiss.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
sst_oiss_2 = sst_oiss_2.chunk(chunks=None)
sst_oiss_2 = xsf.detrend(sst_oiss_2,dim='time',type='linear')


In [7]:
sst_oiss_atl3_1 = Atools.data_sub(sst_oiss_1,-20,0,-3,3).mean(dim='lon').mean(dim='lat')
ssta_oiss_atl3_1,_ = Atools.ano_norm_t(sst_oiss_atl3_1.load())
std_ssta_oiss_atl3_jja_1 = ssta_oiss_atl3_1.sel(time=is_jja(
    ssta_oiss_atl3_1['time.month'])).std(dim='time')

sst_oiss_atl3_2 = Atools.data_sub(sst_oiss_2,-20,0,-3,3).mean(dim='lon').mean(dim='lat')
ssta_oiss_atl3_2,_ = Atools.ano_norm_t(sst_oiss_atl3_2.load())
std_ssta_oiss_atl3_jja_2 = ssta_oiss_atl3_2.sel(time=is_jja(
    ssta_oiss_atl3_2['time.month'])).std(dim='time')


# HadI-SST

In [8]:
## Load data ##
nc = xr.open_mfdataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/HADI_SST/HadISST_sst.nc')
sst_hadi = nc.sst[:]
sst_hadi = sst_hadi.where(sst_hadi>-100)

## take the period 1982/01- 1999/12 ##
sst_hadi_1 = sst_hadi.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_hadi_1 = sst_hadi_1.chunk(chunks=None)
sst_hadi_1 = xsf.detrend(sst_hadi_1,dim='time',type='linear')
## take the period 2000/01- 2014/12 ##
sst_hadi_2 = sst_hadi.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
sst_hadi_2 = sst_hadi_2.chunk(chunks=None)
sst_hadi_2 = xsf.detrend(sst_hadi_2,dim='time',type='linear')

In [9]:
sst_hadi_atl3_1 = Atools.data_sub(sst_hadi_1,-20,0,-3,3).mean(dim='longitude').mean(dim='latitude')
ssta_hadi_atl3_1,_ = Atools.ano_norm_t(sst_hadi_atl3_1.load())
std_ssta_hadi_atl3_jja_1 = ssta_hadi_atl3_1.sel(time=is_jja(
    ssta_hadi_atl3_1['time.month'])).std(dim='time')
sst_hadi_atl3_2 = Atools.data_sub(sst_hadi_2,-20,0,-3,3).mean(dim='longitude').mean(dim='latitude')
ssta_hadi_atl3_2,_ = Atools.ano_norm_t(sst_hadi_atl3_2.load())
std_ssta_hadi_atl3_jja_2 = ssta_hadi_atl3_2.sel(time=is_jja(
    ssta_hadi_atl3_2['time.month'])).std(dim='time')

# Load CMIP data

In [10]:
## created by /Users/aprigent/Documents/Thesis_GEOMAR/Projects/
#weakened_sst_variability_CMIP5/scripts/new_version/to_publish/scripts_tmp/
#create_file_std_ssta_uasa_CMIP5.ipynb
#create_file_std_ssta_uasa_CMIP6.ipynb


## Load CMIP6 period 1982-01 - 1999/12 ##
ssta_atl3_hist_cmip6_1982_1999 = xr.open_dataset(path_data+'ssta_hist_monthly_CMIP6_1982_1999.nc')
ssta_atl3_hist_cmip6_1982_1999_jja = ssta_atl3_hist_cmip6_1982_1999.ssta.sel(time=is_jja(
    ssta_atl3_hist_cmip6_1982_1999['time.month'])).std(dim='time')


ssta_atl3_hist_cmip6_2000_2014 = xr.open_dataset(path_data+'ssta_hist_monthly_CMIP6_2000_2014.nc')
ssta_atl3_hist_cmip6_2000_2014_jja = ssta_atl3_hist_cmip6_2000_2014.ssta.sel(time=is_jja(
    ssta_atl3_hist_cmip6_2000_2014['time.month'])).std(dim='time')


ssta_atl3_hist_cmip5 = xr.open_dataset(path_data+'ssta_hist_monthly_CMIP5.nc')
ssta_atl3_ssp5_cmip5 = xr.open_dataset(path_data+'ssta_ssp5_monthly_CMIP5.nc')

ssta_atl3_hist_cmip6 = xr.open_dataset(path_data+'ssta_hist_monthly_CMIP6.nc')
ssta_atl3_ssp5_cmip6 = xr.open_dataset(path_data+'ssta_ssp585_monthly_CMIP6.nc')


std_ssta_atl3_hist_cmip5_jja = ssta_atl3_hist_cmip5.ssta.sel(time=is_jja(
    ssta_atl3_hist_cmip5['time.month'])).std(dim='time')

std_ssta_atl3_ssp5_cmip5_jja = ssta_atl3_ssp5_cmip5.ssta.sel(time=is_jja(
    ssta_atl3_ssp5_cmip5['time.month'])).std(dim='time')

std_ssta_atl3_hist_cmip6_jja = ssta_atl3_hist_cmip6.ssta.sel(time=is_jja(
    ssta_atl3_hist_cmip6['time.month'])).std(dim='time')

std_ssta_atl3_ssp5_cmip6_jja = ssta_atl3_ssp5_cmip6.ssta.sel(time=is_jja(
    ssta_atl3_ssp5_cmip6['time.month'])).std(dim='time')

In [11]:
print('ens mean CMIP6 1950-1999 =',np.round(std_ssta_atl3_hist_cmip6_jja.mean(dim='model').values,2),'+/-',
      np.round(std_ssta_atl3_hist_cmip6_jja.std(dim='model').values,2))

print('ens mean CMIP6 1982-1999 =',np.round(ssta_atl3_hist_cmip6_1982_1999_jja.mean(dim='model').values,2),'+/-',
      np.round(ssta_atl3_hist_cmip6_1982_1999_jja.std(dim='model').values,2))

print('ens mean CMIP6 2000-2014 =',np.round(ssta_atl3_hist_cmip6_2000_2014_jja.mean(dim='model').values,2),'+/-',
      np.round(ssta_atl3_hist_cmip6_1982_1999_jja.std(dim='model').values,2))

print('ens mean CMIP6 2050-2099 =',np.round(std_ssta_atl3_ssp5_cmip6_jja.mean(dim='model').values,2),'+/-',
      np.round(std_ssta_atl3_ssp5_cmip6_jja.std(dim='model').values,2))

ens mean CMIP6 1950-1999 = 0.51 +/- 0.14
ens mean CMIP6 1982-1999 = 0.49 +/- 0.12
ens mean CMIP6 2000-2014 = 0.48 +/- 0.12
ens mean CMIP6 2050-2099 = 0.42 +/- 0.14


In [12]:
ens_obs_jja_1 = [std_ssta_ersst_atl3_jja_1.values,std_ssta_oiss_atl3_jja_1.values,
           std_ssta_hadi_atl3_jja_1.values,std_ssta_era5_atl3_jja_1.values]

ens_obs_jja_2 = [std_ssta_ersst_atl3_jja_2.values,std_ssta_oiss_atl3_jja_2.values,
           std_ssta_hadi_atl3_jja_2.values,std_ssta_era5_atl3_jja_2.values]

In [13]:
print('OBS ens mean 1982-1999 = ',np.round(np.mean(ens_obs_jja_1),2),'+/-',np.round(np.std(ens_obs_jja_1),2))
print('OBS ens mean 2000-2014 = ',np.round(np.mean(ens_obs_jja_2),2),'+/-',np.round(np.std(ens_obs_jja_2),2))

OBS ens mean 1982-1999 =  0.62 +/- 0.04
OBS ens mean 2000-2014 =  0.42 +/- 0.03
